## **Análisis de Impacto Post-T-MEC**

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

----

**Cargar conjunto de datos**

In [3]:
df_2012_2023 = pd.read_csv('../data/processed/mex_trade_2012_2023_clean.csv')
df_2012_2023.head(5)

,prod_est,coverage,type,year,month,concept,value_usd,status
0,Trade Balance of Goods of Mexico,National,Exports,2012,1,Total Exports,"27,281.58",Final Figures
1,Trade Balance of Goods of Mexico,National,Exports,2012,1,Petroleum,"4,628.30",Final Figures
2,Trade Balance of Goods of Mexico,National,Exports,2012,1,Crude Oil,"4,008.11",Final Figures
3,Trade Balance of Goods of Mexico,National,Exports,2012,1,Other Oil,620.18,Final Figures
4,Trade Balance of Goods of Mexico,National,Exports,2012,1,Non Petroleum,"22,653.28",Final Figures


----


**Dividir el conjunto de datos en dos periodos:**
- **Pre-T-MEC**: 2012–2019
- **Post-T-MEC**: 2020–2023

In [5]:
df_pre_tmec = df_2012_2023[df_2012_2023['year'] <= 2019].copy()
df_post_tmec = df_2012_2023[df_2012_2023['year'] >= 2020].copy()

print(f'Número de registros Pre-T-MEC (2012-2019): {len(df_pre_tmec)}')
print(f'Número de registros Post-T-MEC (2020-2023): {len(df_post_tmec)}')
print('\n')

Número de registros Pre-T-MEC (2012-2019): 1728
Número de registros Post-T-MEC (2020-2023): 828




**Comparar medias de Exportación e Importación**

In [6]:
mean_pre_tmec_exports = df_pre_tmec[df_pre_tmec['type'] == 'Exports']['value_usd'].mean()
mean_pre_tmec_imports = df_pre_tmec[df_pre_tmec['type'] == 'Imports']['value_usd'].mean()

mean_post_tmec_exports = df_post_tmec[df_post_tmec['type'] == 'Exports']['value_usd'].mean()
mean_post_tmec_imports = df_post_tmec[df_post_tmec['type'] == 'Imports']['value_usd'].mean()

print("Medias de Value_USD (Pre-T-MEC):")
print(f"  Exportaciones: {mean_pre_tmec_exports:,.2f}")
print(f"  Importaciones: {mean_pre_tmec_imports:,.2f}")
print("\nMedias de Value_USD (Post-T-MEC):")
print(f"  Exportaciones: {mean_post_tmec_exports:,.2f}")
print(f"  Importaciones: {mean_post_tmec_imports:,.2f}")
print("\n")

Medias de Value_USD (Pre-T-MEC):
  Exportaciones: 12,589.68
  Importaciones: 17,281.85

Medias de Value_USD (Post-T-MEC):
  Exportaciones: 16,168.75
  Importaciones: 22,078.47




**Prueba de Hipótesis**
- H0: No hay diferencia en el valor medio antes y después.
- H1: Sí hay diferencia significativa.

In [8]:
# Nivel de significancia
alpha = 0.05

In [9]:
# --- COMPARACIÓN PARA EXPORTACIONES ---
print("\n--- Comparación para EXPORTACIONES ---")
exports_pre_tmec = df_pre_tmec[df_pre_tmec['type'] == 'Exports']['value_usd'].dropna()
exports_post_tmec = df_post_tmec[df_post_tmec['type'] == 'Exports']['value_usd'].dropna()

# 1. Prueba de Levene para igualdad de varianzas (para decidir entre t-student variantes)
# Asegurarse de tener suficientes datos para la prueba de Levene
if len(exports_pre_tmec) > 1 and len(exports_post_tmec) > 1:
    levene_stat_exp, p_levene_exp = stats.levene(exports_pre_tmec, exports_post_tmec)
    print(f"  Prueba de Levene (Exportaciones): Estadístico={levene_stat_exp:.4f}, p-valor={p_levene_exp:.4f}")

    equal_variances_exp = p_levene_exp >= alpha
    if equal_variances_exp:
        print("  -> Las varianzas de Exportaciones son aproximadamente iguales.")
    else:
        print("  -> Las varianzas de Exportaciones son significativamente diferentes.")

    # 2. Prueba de Normalidad (para decidir entre t-student y Mann-Whitney U)
    # Una forma sencilla es con Shapiro-Wilk (asume N < 5000)
    # Para grandes N, un gráfico Q-Q o histograma es mejor, o simplemente asumir no-normalidad si son datos reales de mercado
    # stats.shapiro puede ser muy sensible a grandes N, incluso para pequeñas desviaciones de normalidad.
    # Por simplicidad, aquí lo usamos, pero en un caso real, considera alternativas.
    
    # Check for enough data for Shapiro-Wilk
    if len(exports_pre_tmec) > 3 and len(exports_post_tmec) > 3:
        shapiro_pre_exp_stat, p_shapiro_pre_exp = stats.shapiro(exports_pre_tmec)
        shapiro_post_exp_stat, p_shapiro_post_exp = stats.shapiro(exports_post_tmec)
        print(f"  Normalidad Exportaciones Pre-T-MEC (Shapiro): p-valor={p_shapiro_pre_exp:.4f}")
        print(f"  Normalidad Exportaciones Post-T-MEC (Shapiro): p-valor={p_shapiro_post_exp:.4f}")

        # Si ambos p-valores son > alpha, asumimos normalidad
        normal_dist_exp = (p_shapiro_pre_exp >= alpha) and (p_shapiro_post_exp >= alpha)
        if normal_dist_exp:
            print("  -> Ambos grupos de Exportaciones parecen seguir una distribución normal.")
        else:
            print("  -> Al menos un grupo de Exportaciones NO parece seguir una distribución normal.")
    else:
        print("  -> No hay suficientes datos para la prueba de normalidad de Shapiro-Wilk en Exportaciones.")
        normal_dist_exp = False # Assume non-normal if not enough data for Shapiro

    # 3. Aplicar prueba de hipótesis adecuada
    if normal_dist_exp: # Si los datos son normales
        if equal_variances_exp:
            # T-test para muestras independientes con varianzas iguales (Student's t-test)
            t_stat_exp, p_value_exp = stats.ttest_ind(exports_pre_tmec, exports_post_tmec, equal_var=True)
            test_type_exp = "T-test (varianzas iguales)"
        else:
            # T-test para muestras independientes con varianzas desiguales (Welch's t-test)
            t_stat_exp, p_value_exp = stats.ttest_ind(exports_pre_tmec, exports_post_tmec, equal_var=False)
            test_type_exp = "T-test (varianzas desiguales - Welch)"
        print(f"  {test_type_exp} (Exportaciones): Estadístico={t_stat_exp:.4f}, p-valor={p_value_exp:.4f}")

    else: # Si los datos NO son normales (o no hay suficientes para Shapiro)
        # Prueba de Mann-Whitney U (no paramétrica, no asume normalidad)
        u_stat_exp, p_value_exp = stats.mannwhitneyu(exports_pre_tmec, exports_post_tmec)
        test_type_exp = "Mann-Whitney U"
        print(f"  {test_type_exp} (Exportaciones): Estadístico={u_stat_exp:.4f}, p-valor={p_value_exp:.4f}")
    
    # Conclusión
    if p_value_exp < alpha:
        print(f"  CONCLUSIÓN: Rechazamos H0. Hay una diferencia significativa en la media de Exportaciones entre los periodos Pre-T-MEC y Post-T-MEC.")
    else:
        print(f"  CONCLUSIÓN: No rechazamos H0. No hay evidencia de una diferencia significativa en la media de Exportaciones entre los periodos Pre-T-MEC y Post-T-MEC.")
else:
    print("  No hay suficientes datos de Exportaciones para realizar pruebas de hipótesis.")



--- Comparación para EXPORTACIONES ---
  Prueba de Levene (Exportaciones): Estadístico=11.9396, p-valor=0.0006
  -> Las varianzas de Exportaciones son significativamente diferentes.
  Normalidad Exportaciones Pre-T-MEC (Shapiro): p-valor=0.0000
  Normalidad Exportaciones Post-T-MEC (Shapiro): p-valor=0.0000
  -> Al menos un grupo de Exportaciones NO parece seguir una distribución normal.
  Mann-Whitney U (Exportaciones): Estadístico=120006.0000, p-valor=0.0000
  CONCLUSIÓN: Rechazamos H0. Hay una diferencia significativa en la media de Exportaciones entre los periodos Pre-T-MEC y Post-T-MEC.


In [10]:
# --- COMPARACIÓN PARA EXPORTACIONES ---
print("\n--- Comparación para EXPORTACIONES ---")
exports_pre_tmec = df_pre_tmec[df_pre_tmec['type'] == 'Exports']['value_usd'].dropna()
exports_post_tmec = df_post_tmec[df_post_tmec['type'] == 'Exports']['value_usd'].dropna()

# 1. Prueba de Levene para igualdad de varianzas (para decidir entre t-student variantes)
# Asegurarse de tener suficientes datos para la prueba de Levene
if len(exports_pre_tmec) > 1 and len(exports_post_tmec) > 1:
    levene_stat_exp, p_levene_exp = stats.levene(exports_pre_tmec, exports_post_tmec)
    print(f"  Prueba de Levene (Exportaciones): Estadístico={levene_stat_exp:.4f}, p-valor={p_levene_exp:.4f}")

    equal_variances_exp = p_levene_exp >= alpha
    if equal_variances_exp:
        print("  -> Las varianzas de Exportaciones son aproximadamente iguales.")
    else:
        print("  -> Las varianzas de Exportaciones son significativamente diferentes.")

    # 2. Prueba de Normalidad (para decidir entre t-student y Mann-Whitney U)
    # Una forma sencilla es con Shapiro-Wilk (asume N < 5000)
    # Para grandes N, un gráfico Q-Q o histograma es mejor, o simplemente asumir no-normalidad si son datos reales de mercado
    # stats.shapiro puede ser muy sensible a grandes N, incluso para pequeñas desviaciones de normalidad.
    # Por simplicidad, aquí lo usamos, pero en un caso real, considera alternativas.
    
    # Check for enough data for Shapiro-Wilk
    if len(exports_pre_tmec) > 3 and len(exports_post_tmec) > 3:
        shapiro_pre_exp_stat, p_shapiro_pre_exp = stats.shapiro(exports_pre_tmec)
        shapiro_post_exp_stat, p_shapiro_post_exp = stats.shapiro(exports_post_tmec)
        print(f"  Normalidad Exportaciones Pre-T-MEC (Shapiro): p-valor={p_shapiro_pre_exp:.4f}")
        print(f"  Normalidad Exportaciones Post-T-MEC (Shapiro): p-valor={p_shapiro_post_exp:.4f}")

        # Si ambos p-valores son > alpha, asumimos normalidad
        normal_dist_exp = (p_shapiro_pre_exp >= alpha) and (p_shapiro_post_exp >= alpha)
        if normal_dist_exp:
            print("  -> Ambos grupos de Exportaciones parecen seguir una distribución normal.")
        else:
            print("  -> Al menos un grupo de Exportaciones NO parece seguir una distribución normal.")
    else:
        print("  -> No hay suficientes datos para la prueba de normalidad de Shapiro-Wilk en Exportaciones.")
        normal_dist_exp = False # Assume non-normal if not enough data for Shapiro

    # 3. Aplicar prueba de hipótesis adecuada
    if normal_dist_exp: # Si los datos son normales
        if equal_variances_exp:
            # T-test para muestras independientes con varianzas iguales (Student's t-test)
            t_stat_exp, p_value_exp = stats.ttest_ind(exports_pre_tmec, exports_post_tmec, equal_var=True)
            test_type_exp = "T-test (varianzas iguales)"
        else:
            # T-test para muestras independientes con varianzas desiguales (Welch's t-test)
            t_stat_exp, p_value_exp = stats.ttest_ind(exports_pre_tmec, exports_post_tmec, equal_var=False)
            test_type_exp = "T-test (varianzas desiguales - Welch)"
        print(f"  {test_type_exp} (Exportaciones): Estadístico={t_stat_exp:.4f}, p-valor={p_value_exp:.4f}")

    else: # Si los datos NO son normales (o no hay suficientes para Shapiro)
        # Prueba de Mann-Whitney U (no paramétrica, no asume normalidad)
        u_stat_exp, p_value_exp = stats.mannwhitneyu(exports_pre_tmec, exports_post_tmec)
        test_type_exp = "Mann-Whitney U"
        print(f"  {test_type_exp} (Exportaciones): Estadístico={u_stat_exp:.4f}, p-valor={p_value_exp:.4f}")
    
    # Conclusión
    if p_value_exp < alpha:
        print(f"  CONCLUSIÓN: Rechazamos H0. Hay una diferencia significativa en la media de Exportaciones entre los periodos Pre-T-MEC y Post-T-MEC.")
    else:
        print(f"  CONCLUSIÓN: No rechazamos H0. No hay evidencia de una diferencia significativa en la media de Exportaciones entre los periodos Pre-T-MEC y Post-T-MEC.")
else:
    print("  No hay suficientes datos de Exportaciones para realizar pruebas de hipótesis.")



--- Comparación para EXPORTACIONES ---
  Prueba de Levene (Exportaciones): Estadístico=11.9396, p-valor=0.0006
  -> Las varianzas de Exportaciones son significativamente diferentes.
  Normalidad Exportaciones Pre-T-MEC (Shapiro): p-valor=0.0000
  Normalidad Exportaciones Post-T-MEC (Shapiro): p-valor=0.0000
  -> Al menos un grupo de Exportaciones NO parece seguir una distribución normal.
  Mann-Whitney U (Exportaciones): Estadístico=120006.0000, p-valor=0.0000
  CONCLUSIÓN: Rechazamos H0. Hay una diferencia significativa en la media de Exportaciones entre los periodos Pre-T-MEC y Post-T-MEC.
